In [1]:
#!/usr/bin/env python
# -*-coding:utf-8 -*-
'''
@File    :   library_functions_with_grouping.ipynb
@Time    :   2023/07/05 00:02:49
@Author  :   Asra Aslam 
@Version :   1.0
@Contact :   a.aslam2@leeds.ac.uk
@License :   (C)Copyright Asra Aslam DynAIRX
@Desc    :   This jupyter notebook will create groupings of diseases specified in list "simple_conditions_list"
             Presently gruping is based on simple search function, more difficult functions will get added on next verisons.
             It consider efi and Lauren file to retrive conditions data
             Generated groupings of different conditions goes in output folder
'''

import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

In [2]:
filepath_database_LW="output/Snomed_Generated_for_Baseline1_codelist.csv" #---Lauren file for grouping----
filepath_database_efi="mappings_and_baselines/Baseline2_Codelist.csv" #---efi file for grouping----
filepath_grouped_database_SNOMED="output/codelists/"

# Analyzing Our Generated SNOMED codes file for Lauren's code file

In [3]:
df_database_LW = pd.read_csv(filepath_database_LW, dtype={'SnomedCTConceptId': str}) 
df_database_LW

,CleansedReadCode,Term,Disease,SnomedCTConceptId
0,7A11000,Emerg repl aneurysm bifurc aorta by anast aort...,Abdominal Aortic Aneurysm,175283003
1,7A11211,Y graft of abdominal Aortic aneurysm (emergency),Abdominal Aortic Aneurysm,233374003
2,7A13.00,Emergency replacement of aneurysmal segment of...,Abdominal Aortic Aneurysm,175297006
3,7A13.11,Emergency repair of aortic aneurysm,Abdominal Aortic Aneurysm,175297006
4,7A13411,Tube graft abdominal Aortic aneurysm (emergency),Abdominal Aortic Aneurysm,233377005
...,...,...,...,...
5719,C32y.00,Other disorders of lipoid metabolism,Lipid Disorder,267431006
5720,C32yz00,Other disorder of lipoid metabolism NOS,Lipid Disorder,267431006
5721,C32z.00,Disorder of lipoid metabolism NOS,Lipid Disorder,267431006
5722,Cyu8D00,[X]Other hyperlipidaemia,Lipid Disorder,55822004


In [4]:
df_database_LW.dtypes

CleansedReadCode     object
Term                 object
Disease              object
SnomedCTConceptId    object
dtype: object

# Analyzing Our efi file that is already with SNOMED codes

In [5]:
df_database_efi = pd.read_csv(filepath_database_efi, dtype={'SNOMEDCT_CONCEPTID': str})  #pd.read_excel(filepath_database_efi, sheet_name=0)
df_database_efi

,Deficit,SNOMEDCT_CONCEPTID,CTV3,Provenance,Codedescription,TimeConstraintyears,AgeLimit,Otherinstructions
0,Abdominal pain,162031009,195..,efi-abdominal_pain,Indigestion symptoms,5.0,NaN,NaN
1,Abdominal pain,16331000,1955,efi-abdominal_pain,Heartburn,5.0,NaN,NaN
2,Abdominal pain,247358007,196..,efi-abdominal_pain,Type of GIT pain (& [abdominal pain] or [sympt...,5.0,NaN,NaN
3,Abdominal pain,270477000,196..,efi-abdominal_pain,Type of GIT pain (& [abdominal pain] or [sympt...,5.0,NaN,NaN
4,Abdominal pain,162038003,1963,efi-abdominal_pain,Non-colicky abdominal pain,5.0,NaN,NaN
...,...,...,...,...,...,...,...,...
7550,Weight loss,401003006,XaIxC,efi-weight_loss,Weight loss from baseline weight,5.0,NaN,NaN
7551,Weight loss,198511000000103,XaKwR,efi-weight_loss,Complaining of weight loss,NaN,NaN,NaN
7552,Weight loss,422868009,XaQgK,efi-weight_loss,Unexplained weight loss,NaN,NaN,NaN
7553,Weight loss,448765001,XaXTs,efi-weight_loss,Unintentional weight loss,NaN,NaN,NaN


In [6]:
df_database_efi.dtypes

Deficit                 object
SNOMEDCT_CONCEPTID      object
CTV3                    object
Provenance              object
Codedescription         object
TimeConstraintyears    float64
AgeLimit               float64
Otherinstructions       object
dtype: object

In [7]:
# rename SNOMEDCT_CONCEPTID, Deficit and Codedescription column in efi list same as LW list
df_database_efi.rename(columns={'SNOMEDCT_CONCEPTID':'SnomedCTConceptId'}, inplace=True)
df_database_efi.rename(columns={'Codedescription':'Term'}, inplace=True)
df_database_efi.rename(columns={'Deficit':'Disease'}, inplace=True)
df_database_efi.columns

Index(['Disease', 'SnomedCTConceptId', 'CTV3', 'Provenance', 'Term',
       'TimeConstraintyears', 'AgeLimit', 'Otherinstructions'],
      dtype='object')

In [8]:
# add column of Otherinstructions to Lauren's dataframe as we may use "Otherinstructions" column after grouping
df_database_LW["Otherinstructions"] = np.nan
df_database_LW.columns

Index(['CleansedReadCode', 'Term', 'Disease', 'SnomedCTConceptId',
       'Otherinstructions'],
      dtype='object')

# Search function (s)

In [9]:
def search_condition(df_database, condition_name): #--retrive rows in database where condition occured---
    
    rows=df_database[df_database['Disease'] == condition_name]  #rows=df_database[(df_database == condition_name).any(axis=1)] #--all code below can be replaced with this single line, but case ignore will not work with this operation, and we dont have search in full database, so could make efficient only by fusing on specific columns---
    
    return rows.sort_index()

# Processing Conditions for Grouping
## 1. Conditions which are simple and need searching only
* Grouping of these conditions will require only use of search functions. Examples include Anxiety, Heart Failure etc.

In [10]:
# Provide list of conditions
simple_conditions_list= ['Abdominal Aortic Aneurysm', 'Abdominal Hernia', 'Actinic keratosis', 'Allergic And Chronic rhinitis', 'Ankylosing Spondylitis', 'Anorectal Prolapse', 'Anxiety', 'Attention Deficit Hyperactivity Disorder', 'Autism and Asperger Syndrome', 'Barrett Oesophagus', 'Bronchiectasis', 'Carcinoma in situ_Cervical', 'Cerebral palsy', 'Chronic Dermatitis Eczema', 'Chronic Sinusitis', 'Chronic Tinnitus', 'Chronic Urticaria', 'Coeliac Disease', 'Colonic polyp', 'Cystic Fibrosis', 'Dermatitis atopic contact', 'Dilated Cardiomyopathy', 'Diverticular Disease', 'Down Syndrome', 'Dysmenorrhoea', 'Endometrial Hyperplasia and Hypertrophy', 'Endometriosis', 'Fatty Liver', 'Gastrooesophageal Reflux Disease', 'Gout', 'Heart Failure', 'HIV', 'Hypertrophic Cardiomyopathy', 'Irritable Bowel Syndrome', 'Lupus Erythematosus', 'Menorrhagia and Polymenorrhea', 'Multiple Sclerosis', 'Myasthenia Gravis', 'Obstructive Sleep Apnoea', 'Peripheral Neuropathies Excluding Cranial nerve and carpal tunnel syndrome', 'Personality Disorder', 'Pituitary adenoma', 'Polycythaemia vera', 'Psoriasis', 'Raynaud Syndrome', 'Recurrent pulmonary embolus', 'Renal Stones', 'Rheumatoid Arthritis', 'Rosacea', 'Sarcoidosis', 'Scoliosis', 'Secondary Polycythaemia', 'Sick sinus Syndrome', 'Sjogren Disease', 'Spinal stenosis', 'Splenomegaly', 'Spondylolisthesis', 'Systemic Sclerosis', 'Tuberculosis', 'Urolithiasis', 'Uterovaginal Genital Prolapse', 'Vitiligo', 'Inflammatory arthritis', 'Problems managing finances', 'Washing & bathing problems', 'Dressing & grooming problems', 'Housebound', 'Mobility problems', 'Smoker (ex)', 'Falls', 'Foot problems', 'Skin ulcer', 'Palliative care', 'Hypotension / syncope', 'Medication management problems', 'Smoker (current)', 'Fragility fracture', 'Stress', 'Chronic Fatigue Syndrome', 'Faecal incontinence', 'Requirement for care', 'Sleep problems', 'Dizziness', 'Dyspnoea', 'Back pain', 'Headache', 'Social vulnerability', 'Meal preparation problems', 'Toileting problems', 'Memory concerns', 'Weakness', 'Weight loss', 'Fracture', 'Urinary system disease', 'Environment problems', 'Shopping problems', 'Self-harm', 'Musculoskeletal problems', 'Acne', 'Asbestosis', 'Cholelithiasis', 'Chronic Pancreatitis', 'Gastritis and Duodenitis', 'Hyperparathyroidism', 'Hyposplenism', 'Meniere Disease', 'Polycystic Ovarian Syndrome', 'Polymyalgia Rheumatica', 'Primary Idiopathic Thrombocytopaenia', 'PRIMARYPULMONARYHypertension', 'Prostate Disorder', 'Psoriatic Arthropathy', 'Psychoactive Substance Misuse', 'Pulmonary Fibrosis', 'SECONDARYPULMONARYHypertension', 'Sickle cell anaemia', 'Spina bifida', 'Spondylosis', 'Subdural Haematoma no-traumatic', 'Thalassaemia', 'Thrombophilia', 'Thyroid Problem']
# ['Body mass index']
# ['Obesity']
# ['Housebound']
# ['Activity limitation']
# ['Glaucoma']
# simple_conditions_list= ['Peptic ulcer Disease']
# ['Peptic ulcer disease']
# ['Epilepsy']
# ['Seizures']
# ['Irritable Bowel Syndrome']
# ['Inflammatory bowel disease']
# ['Inflammatory Bowel Disease Ulcerative Colitis']
# ['Inflammatory Bowel Disease Crohn Disease']
# ['Visual Impairment and Blindness']
# ['Glomerulonephritis']
# ['Visual impairment']
#['Cataract']
# ['Anterior and Intermediate Uveitis']
# ['Macular Degeneration']
#['Alcohol Problem']
#['Chronic Liver disease And Viral hepatitis']
# ['Autoimmune liver Disease']
#['Alcoholic Liver Disease']
#['Oesophageal varices']
#['Alcohol-related Brain Injury']
#[]'Anxiety', 'Heart Failure']

In [11]:
# For each condition do Searching, grouping, and saving
for condition_name in simple_conditions_list:

    # search
    rows_LW_with_condition=search_condition(df_database_LW, condition_name) #--get rows matching with condition in Lauren's file
    rows_efi_with_condition=search_condition(df_database_efi, condition_name) #--get rows matching with condition in efi file

    print("{} Rows Matching with Condition {} in Lauren's file".format(len(rows_LW_with_condition),condition_name))
    print("{} Rows Matching with Condition {} in efi file".format(len(rows_efi_with_condition),condition_name))

    # group
    to_group_snomed_LW=rows_LW_with_condition[['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions']] #--considering only specific columns for grouping--
    to_group_snomed_efi=rows_efi_with_condition[['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions']] #--considering only specific columns for grouping--
    cols_in_shortlisted_rows = to_group_snomed_LW.columns.to_list() # Note: to_group_snomed_LW and rows_efi_with_condition have same columns; Store columns before adding `origin` column to use later in groupby
    to_group_snomed_LW['origin'] = 'Lauren_codelist'
    to_group_snomed_efi['origin'] = 'efi'

    print(to_group_snomed_efi.columns)
    print(to_group_snomed_LW.columns)
    # grouped_rows_for_condition = pd.concat([to_group_snomed_efi, to_group_snomed_LW], sort=True).drop_duplicates() # conctatenate to put all rows in a group--
    grouped_rows_for_condition = pd.concat([to_group_snomed_LW, to_group_snomed_efi]).reset_index(drop=True).groupby(by = cols_in_shortlisted_rows, axis=0, dropna=False).apply(lambda x: x['origin'].values).to_frame('origin').reset_index() # conctatenate to put all rows in a group while preserving origin--

    
    # save
    if condition_name.__contains__('/'): # this if is just to deal with condition name "Hypotension / syncope"--
        condition_name = condition_name.replace("/", "_" )
    filepath_to_save=filepath_grouped_database_SNOMED+''+condition_name+'.csv'
    print("Your grouped file for condition {} is saved at {}".format(condition_name, filepath_to_save))
    grouped_rows_for_condition.to_csv(filepath_to_save, index=False)
    

11 Rows Matching with Condition Abdominal Aortic Aneurysm in Lauren's file
0 Rows Matching with Condition Abdominal Aortic Aneurysm in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Abdominal Aortic Aneurysm is saved at output/codelists/Abdominal Aortic Aneurysm.csv
192 Rows Matching with Condition Abdominal Hernia in Lauren's file
0 Rows Matching with Condition Abdominal Hernia in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Abdominal Hernia is saved at output/codelists/Abdominal Hernia.csv
4 Rows Matching with Condition Actinic keratosis in Lauren's file
0 Rows Matching with Condition Actinic keratosis in efi file
Index(

C:\Users\hssaas\AppData\Local\Temp\ipykernel_16284\4253553438.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_group_snomed_efi['origin'] = 'efi'


24 Rows Matching with Condition Carcinoma in situ_Cervical in Lauren's file
0 Rows Matching with Condition Carcinoma in situ_Cervical in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Carcinoma in situ_Cervical is saved at output/codelists/Carcinoma in situ_Cervical.csv
31 Rows Matching with Condition Cerebral palsy in Lauren's file
0 Rows Matching with Condition Cerebral palsy in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Cerebral palsy is saved at output/codelists/Cerebral palsy.csv
45 Rows Matching with Condition Chronic Dermatitis Eczema in Lauren's file
0 Rows Matching with Condition Chronic Dermatitis Eczema in efi

C:\Users\hssaas\AppData\Local\Temp\ipykernel_16284\4253553438.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_group_snomed_efi['origin'] = 'efi'


Your grouped file for condition Personality Disorder is saved at output/codelists/Personality Disorder.csv
1 Rows Matching with Condition Pituitary adenoma in Lauren's file
0 Rows Matching with Condition Pituitary adenoma in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Pituitary adenoma is saved at output/codelists/Pituitary adenoma.csv
5 Rows Matching with Condition Polycythaemia vera in Lauren's file
0 Rows Matching with Condition Polycythaemia vera in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Polycythaemia vera is saved at output/codelists/Polycythaemia vera.csv
32 Rows Matching with Condition Psoriasis in Lauren's

...

0 Rows Matching with Condition Smoker (current) in Lauren's file
37 Rows Matching with Condition Smoker (current) in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Smoker (current) is saved at output/codelists/Smoker (current).csv
0 Rows Matching with Condition Fragility fracture in Lauren's file
365 Rows Matching with Condition Fragility fracture in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Fragility fracture is saved at output/codelists/Fragility fracture.csv
0 Rows Matching with Condition Stress in Lauren's file
43 Rows Matching with Condition Stress in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherin

C:\Users\hssaas\AppData\Local\Temp\ipykernel_16284\4253553438.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_group_snomed_efi['origin'] = 'efi'
C:\Users\hssaas\AppData\Local\Temp\ipykernel_16284\4253553438.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_group_snomed_efi['origin'] = 'efi'
C:\Users\hssaas\AppData\Local\Temp\ipykernel_16284\4253553438.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

Your grouped file for condition Back pain is saved at output/codelists/Back pain.csv
0 Rows Matching with Condition Headache in Lauren's file
21 Rows Matching with Condition Headache in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Headache is saved at output/codelists/Headache.csv
0 Rows Matching with Condition Social vulnerability in Lauren's file
23 Rows Matching with Condition Social vulnerability in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Social vulnerability is saved at output/codelists/Social vulnerability.csv
0 Rows Matching with Condition Meal preparation problems in Lauren's file
18 Rows Matching with Condi

C:\Users\hssaas\AppData\Local\Temp\ipykernel_16284\4253553438.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_group_snomed_efi['origin'] = 'efi'
C:\Users\hssaas\AppData\Local\Temp\ipykernel_16284\4253553438.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_group_snomed_efi['origin'] = 'efi'
C:\Users\hssaas\AppData\Local\Temp\ipykernel_16284\4253553438.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

20 Rows Matching with Condition Gastritis and Duodenitis in Lauren's file
0 Rows Matching with Condition Gastritis and Duodenitis in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Gastritis and Duodenitis is saved at output/codelists/Gastritis and Duodenitis.csv
12 Rows Matching with Condition Hyperparathyroidism in Lauren's file
0 Rows Matching with Condition Hyperparathyroidism in efi file
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Index(['SnomedCTConceptId', 'Term', 'Disease', 'Otherinstructions', 'origin'], dtype='object')
Your grouped file for condition Hyperparathyroidism is saved at output/codelists/Hyperparathyroidism.csv
11 Rows Matching with Condition Hyposplenism in Lauren's file
0 Rows Matching with Condition Hyposplenism in efi file
Index(['